In [1]:
import sys
import os
sys.path.append(os.path.join(os.getcwd(), ".."))
sys.path.append("../../wrench/")

In [2]:
from rulecleaner_src.lfs_tree import keyword_labelling_func_builder
from rulecleaner_src.TreeRules import SPAM, HAM, ABSTAIN, PredicateNode
from rulecleaner_src.LFRepair import populate_violations, fix_rules_with_solver_input
from rulecleaner_src.utils import run_label_model_with_funcs, select_user_input, clean_text


import re
import psycopg2
import pandas as pd
from snorkel.labeling import (
	LabelingFunction, 
	labeling_function, 
	PandasLFApplier, 
	LFAnalysis,
	filter_unlabeled_dataframe
	)
from snorkel.labeling.model import MajorityLabelVoter, LabelModel
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import matplotlib.patches as mpatches
import pulp
from sklearn.metrics import accuracy_score, classification_report
import random
from collections import deque, defaultdict
import numpy as np
import pickle
from IPython.display import Image, display 
import datetime
import itertools

[nltk_data] Downloading package words to /Users/chenjieli/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/chenjieli/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
from rulecleaner_src.example_tree_rules import (
gen_amazon_funcs,
gen_professor_teacher_funcs,
gen_painter_architecht_funcs,
gen_imdb_funcs,
gen_pj_funcs,
gen_pp_funcs,
gen_yelp_funcs,
gen_plots_funcs,
gen_fakenews_funcs,
gen_dbpedia_funcs,
gen_agnews_funcs,
gen_tweets_funcs,
gen_spam_funcs
)


In [4]:
dataset_dict = {
    "plots": gen_plots_funcs,
    "amazon": gen_amazon_funcs,
    "dbpedia": gen_dbpedia_funcs,
    "agnews": gen_agnews_funcs,
    "physician_professor": gen_pp_funcs,
    "imdb": gen_imdb_funcs,
    "fakenews": gen_fakenews_funcs,
    "yelp": gen_yelp_funcs,
    "photographer_journalist": gen_pj_funcs,
    "professor_teacher": gen_professor_teacher_funcs,
    "painter_architect": gen_painter_architecht_funcs,
    "tweets": gen_tweets_funcs,
    "spam": gen_spam_funcs,
}



In [5]:
from rulecleaner_src.main import main

In [31]:

user_input_sizes = [40]
# random_states = [1,2,3,4,5,6,7,8, 9, 10]
random_states = [1]
lf_acc_threshs = [0.7]
instance_acc_threshs = [0.8]
non_abstain_threshs = [0.8]
datasets = list(dataset_dict)
func_dictionary = [dataset_dict]
instance_acc_on_valids=[False]
use_non_abstains=[False]
# model_types=['dawidskene','flyingsquid']
# model_types = ['majority', 'weighted_majority', ]
model_types = ['fable', ]
pfile_name_prefix = ['additional_other_label_models_april15/']
num_class=[2]



In [32]:
input_params = list(itertools.product(
    user_input_sizes,
    lf_acc_threshs,
    instance_acc_threshs,
    non_abstain_threshs,
    datasets,
    random_states,
    func_dictionary,
    instance_acc_on_valids,
    use_non_abstains,
    pfile_name_prefix,
    ['witan'],
    num_class,
    model_types
))


In [33]:
input_params

[(40,
  0.7,
  0.8,
  0.8,
  'plots',
  1,
  {'plots': <function rulecleaner_src.example_tree_rules.gen_plots_funcs()>,
   'amazon': <function rulecleaner_src.example_tree_rules.gen_amazon_funcs()>,
   'dbpedia': <function rulecleaner_src.example_tree_rules.gen_dbpedia_funcs()>,
   'agnews': <function rulecleaner_src.example_tree_rules.gen_agnews_funcs()>,
   'physician_professor': <function rulecleaner_src.example_tree_rules.gen_pp_funcs()>,
   'imdb': <function rulecleaner_src.example_tree_rules.gen_imdb_funcs()>,
   'fakenews': <function rulecleaner_src.example_tree_rules.gen_fakenews_funcs()>,
   'yelp': <function rulecleaner_src.example_tree_rules.gen_yelp_funcs()>,
   'photographer_journalist': <function rulecleaner_src.example_tree_rules.gen_pj_funcs()>,
   'professor_teacher': <function rulecleaner_src.example_tree_rules.gen_professor_teacher_funcs()>,
   'painter_architect': <function rulecleaner_src.example_tree_rules.gen_painter_architecht_funcs()>,
   'tweets': <function ru

In [34]:
for ip in input_params:
    main(*ip)

user_input_size = 40
lf_acc_thresh = 0.7
instance_acc_thresh = 0.8
min_non_abstain_thresh = 0.8
dataset_name = plots
random_state = 1
funcs_dictionary = {'plots': <function gen_plots_funcs at 0x3335f4c10>, 'amazon': <function gen_amazon_funcs at 0x3335f4820>, 'dbpedia': <function gen_dbpedia_funcs at 0x3335f4d30>, 'agnews': <function gen_agnews_funcs at 0x3335f4dc0>, 'physician_professor': <function gen_pp_funcs at 0x3335f4af0>, 'imdb': <function gen_imdb_funcs at 0x3335f49d0>, 'fakenews': <function gen_fakenews_funcs at 0x3335f4ca0>, 'yelp': <function gen_yelp_funcs at 0x3335f4b80>, 'photographer_journalist': <function gen_pj_funcs at 0x3335f4a60>, 'professor_teacher': <function gen_professor_teacher_funcs at 0x3335f48b0>, 'painter_architect': <function gen_painter_architecht_funcs at 0x3335f4940>, 'tweets': <function gen_tweets_funcs at 0x3335f4e50>, 'spam': <function gen_spam_funcs at 0x3335f4ee0>}
instance_acc_on_valid = False
use_non_abstain = False
pickle_result_file_name_prefix 

/Users/chenjieli/Desktop/RuleCleaner/rulecleaner_src/../rulecleaner_src/utils.py:83: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sentences_df = pd.read_sql(f'SELECT * FROM {dataset_name}', conn)


TypeError: __init__() missing 1 required positional argument: 'kernel_function'